In [1]:
import csv
import numpy as np
#import the data to Python 
with open('/Users/zhejindong/Desktop/EECS6720-hw1-data/X_train.csv') as csvfile:
    readCSV=csv.reader(csvfile,delimiter=',')
    data=[]
    for row in readCSV:
        for i in range(len(row)):
            data.append(float(row[i]))

x_train=np.array(data).reshape([4140,54])

with open('/Users/zhejindong/Desktop/EECS6720-hw1-data/X_test.csv') as csvfile:
    readCSV=csv.reader(csvfile,delimiter=',')
    line=readCSV.line_num
    data=[]
    for row in readCSV:
        for i in range(len(row)):
            data.append(np.float64(row[i]))
            
x_test=np.array(data).reshape([461,54])

with open('/Users/zhejindong/Desktop/EECS6720-hw1-data/label_train.csv') as csvfile:
    readCSV=csv.reader(csvfile,delimiter=',')
    data=[]
    for row in readCSV:
        for i in range(len(row)):
            data.append(float(row[i]))
            
y_train=np.array(data).reshape([4140])

with open('/Users/zhejindong/Desktop/EECS6720-hw1-data/label_test.csv') as csvfile:
    readCSV=csv.reader(csvfile,delimiter=',')
    data=[]
    for row in readCSV:
        for i in range(len(row)):
            data.append(float(row[i]))
            
y_test=np.array(data).reshape([461])


# import the content of emails 
f = open('/Users/zhejindong/Desktop/EECS6720-hw1-data/README')
words=f.read()
words=words.split('\n')
words=words[3:-1]

Each 54-dimensional vector x has a label y with y = 0 indicating “non-spam” and y = 1 indicating
“spam”. We model the nth feature vector of a spam email as 
p(xn| 1, yn = 1) ~ Poisson(xn,d| 1,d), d=1 and similarly for class 0. We model the labels as 
yn ~ Bernoulli(lambda). Assume independent gamma priors on all  1,d and  0,d, as in Problem 3, with a = 1 and b = 1. For the label bias assume the prior ~ Beta(e,f) and set e = f = 1.

In [2]:
#build the bayesian model 
# y2: new y2 under prediction p(y2 = 1|y_train)      p(x∗|y∗ = y, {xi : yi = y})
import scipy
import math
from scipy.special import gamma, factorial
N=y_train.shape[0]
e=1 # parameter of Beta distribution 
f=1 # parameter of Beta distribution 
a=1 # parameter of gamma distribution 
b=1 # parameter of gamma distribution 
np.float64(x_train)
np.float64(x_test)

def log_factor(x):
    log=0
    while x!=0:
        log+=math.log(x)
        x-=1  
    return log

def prob1(x_test):
    prob=0
    p_y2_y_train=(e+sum(y_train))/((N+e+f)*1.0)
    for i in range(54): # calculating the probability of bing a spam email, using log() to avoid overflow 
            gamma_para1=sum(x_train[y_train==1][:,i])+a 
            gamma_para2=sum(x_train[y_train==1][:,i])+a+x_test[i]
            gamma_numerator=0
            gamma_de=0
            while gamma_para1!=1:
                gamma_numerator+=math.log(gamma_para1-1)
                gamma_para1-=1
            while gamma_para2!=1:
                gamma_de+=math.log(gamma_para2-1)
                gamma_para2-=1  
            
            prob=prob+(gamma_de+gamma_para1*math.log(N+b)-gamma_numerator-log_factor(x_test[i])-gamma_para2*math.log(N+b+1))
            
    prob=math.log(p_y2_y_train)+prob          
    return prob

def prob0(x_test):
    prob=0
    p_y2_y_train=(f+y_train.shape[0]-sum(y_train))/((N+e+f)*1.0)
    for i in range(54): # calculating the probability of bing a non-spam email, using log() to avoid overflow 
            #print(i)
            gamma_para1=sum(x_train[y_train==0][:,i])+a
            gamma_para2=sum(x_train[y_train==0][:,i])+a+x_test[i]     
            gamma_numerator=0
            gamma_de=0
            while gamma_para1!=1:
                gamma_numerator+=math.log(gamma_para1-1)
                gamma_para1-=1
        #print('gamma_numerator is',gamma_numerator)
            while gamma_para2!=1:
                gamma_de+=math.log(gamma_para2-1)
                gamma_para2-=1      
                
            prob=prob+(gamma_de+gamma_para1*math.log(N+b)-gamma_numerator-log_factor(x_test[i])-gamma_para2*math.log(N+b+1))
    
    prob+=math.log(p_y2_y_train)
    return prob

def prob1_analysis(x_test):
    prob=np.full((54),-1)
    lamda=np.full((54),-1)
    for i in range(54):
            gamma_para1=sum(x_train[y_train==1][:,i])+a 
            gamma_para2=sum(x_train[y_train==1][:,i])+a+x_test[i]
            lamda[i]=(gamma_para1*1.0)/(b+N)
            gamma_numerator=0
            gamma_de=0
            while gamma_para1!=1:
                gamma_numerator+=math.log(gamma_para1-1)
                gamma_para1-=1
            while gamma_para2!=1:
                gamma_de+=math.log(gamma_para2-1)
                gamma_para2-=1  
            # calculate the log of probability 
            prob[i]=(gamma_de+gamma_para1*math.log(N+b)-gamma_numerator-log_factor(x_test[i])-gamma_para2*math.log(N+b+1))
            
    return [prob,lamda] 

def prob0_analysis(x_test):
    prob=np.full((54),-1)
    lamda=np.full((54),-1)
    for i in range(54):
            gamma_para1=sum(x_train[y_train==0][:,i])+a
            gamma_para2=sum(x_train[y_train==0][:,i])+a+x_test[i]  
            lamda[i]=(gamma_para1*1.0)/(b+N)
            gamma_numerator=0
            gamma_de=0
            while gamma_para1!=1:
                gamma_numerator+=math.log(gamma_para1-1)
                gamma_para1-=1
            while gamma_para2!=1:
                gamma_de+=math.log(gamma_para2-1)
                gamma_para2-=1      
                
            prob[i]=(gamma_de+gamma_para1*math.log(N+b)-gamma_numerator-log_factor(x_test[i])-gamma_para2*math.log(N+b+1))

    return [prob,lamda]

In [3]:
#probability and prediction 
probability=np.full((x_test.shape[0]),-1.0)
prediction=np.full((x_test.shape[0]),-1)
for i in range(x_test.shape[0]):
    probability[i]=prob1(x_test[i])-prob0(x_test[i])
    
for i in range(x_test.shape[0]):
    if probability[i]>0:
        prediction[i]=1
    else:
        prediction[i]=0

In [16]:
probability[7]

260.27190734732835

In [15]:
math.exp(probability[7])

1.0830615930252809e+113

In [13]:
np.argsort(probability)

array([263, 402, 227, 349, 323, 221, 325, 244, 366, 365, 367, 363, 362,
       364, 357, 210, 382, 215, 389, 302, 359, 358, 369, 188, 266, 339,
       196, 272, 297, 269, 229, 212, 293, 259, 242, 375, 400, 201, 222,
       240, 337, 276, 247, 216, 264, 318, 328, 333, 383, 220, 289, 194,
       288, 458, 324, 388, 192, 428, 275, 277, 203, 214, 295, 257, 334,
       384, 246, 393, 224, 186, 399, 316, 270, 267, 311, 332, 352, 347,
       274, 373, 391, 238, 307, 206, 330, 401,  25, 183, 439, 284, 185,
       387, 285, 342, 412, 253, 317, 385, 189, 262, 245, 408, 260, 235,
       379, 381, 271, 343, 398, 394, 396, 380, 374, 432, 299, 208, 309,
       232, 265, 200, 290, 441, 231, 283, 230, 378, 205, 294, 300, 282,
       197, 313, 312, 310, 308, 314, 456, 395, 254, 301, 305, 319, 211,
       195, 351, 340, 338, 209, 452, 202, 451, 346, 442, 354, 386,  87,
       252, 321, 168, 233, 322, 320, 217, 447, 154, 169, 429, 287, 377,
       356, 420, 410, 315, 418, 355, 298, 191, 450, 171, 286, 21

In [4]:
# select the misclassified emails
wrong_index=np.argwhere(y_test!=prediction).reshape((1,42))

In [5]:
# select 10 emails with most ambiguous probability 
np.argsort(abs(probability))[:10]

array([419, 360, 243, 336, 341, 361, 368,  50, 372, 390])

In [6]:
# Pick three misclassified emails and for each email plot its features x compared with E[~ 1] and E[~ 0], 
#and give the predictive probabilities for that email.
import seaborn
from matplotlib import pyplot as plt  
x_axis1=prob1_analysis(x_test[327])[0]
x_axis0=prob0_analysis(x_test[327])[0]
lamda1=prob1_analysis(x_test[327])[1]
lamda0=prob0_analysis(x_test[327])[1]
average_lamda1=np.mean(lamda1)
average_lamda0=np.mean(lamda0)
mistake=seaborn.stripplot(x=words,y=lamda1,color='r')
mistake=seaborn.stripplot(x=words,y=lamda0, color='b')
plt.bar(words,x_test[87],color="gray") 
plt.title('misclassified email_1')
plt.xlabel('words')
plt.ylabel('feature value')
plt.xticks(rotation=90)
plt.savefig('/Users/zhejindong/Desktop/EECS6720-hw1-data/1.jpg')